# **CALIBRADOR**

Toma el SVM pre-entrenado, y se le acopla el scaler, lee un CSV de calibración, el cual no está balanceado, y entrena una capa de calibración (Platt/sigmoid) para convertir los scores del SVM en probabilidades confiables. Guarda todo como svm_calibrado.pkl, listo para predict_proba().

**1. Importación de Librerias**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os, json, joblib
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import log_loss

**2. Rutas de Entrada y Salida, Parametros Necesarios**

In [ ]:
# Carpeta donde se guardará el .pkl calibrado
OUT_DIR = "/content/drive/MyDrive/Colab Notebooks/TESIS/FASE ONLINE/ARCHIVOS ONLINE"

# Artefactos existentes
PATH_SVM_BASE = "/content/drive/MyDrive/Colab Notebooks/TESIS/FASE OFFLINE/MODELOS_BALANCEADOS/svm_modelo_smote_muñeca__50hz.pkl"
PATH_SCALER   = "/content/drive/MyDrive/Colab Notebooks/TESIS/FASE OFFLINE/MODELOS_BALANCEADOS/scaler_svm_smote_muneca_50hz.pkl"
PATH_SCHEMA   =  "/content/drive/MyDrive/Colab Notebooks/TESIS/FASE ONLINE/ARCHIVOS ONLINE/schema.json"
PATH_CALIB_CSV= "/content/drive/MyDrive/Colab Notebooks/TESIS/FASE OFFLINE/DATASET_ETAPAS_POSTERIORES/ventanas_muñeca_50.csv"  #SIN SMOTE (muñeca-50)

#Parametros
LABEL_COL = "etiqueta"
FILTRO_UBICACION = "muñeca"
FILTRO_FREC = 50

# Salida
PATH_OUT_CALIB = os.path.join(OUT_DIR, "svm_calibrado.pkl")

**3. Funciones Necesarias**

In [ ]:
def cargar_features_desde_schema(path_schema: str):
    if not os.path.exists(path_schema):
        raise FileNotFoundError(f"No se encontró schema.json en: {path_schema}")
    with open(path_schema, "r", encoding="utf-8") as f:
        schema = json.load(f)
    feats = schema["features"]
    if not isinstance(feats, list) or len(feats) == 0:
        raise ValueError("schema.json no tiene 'features' válidas.")
    return feats, schema

def armar_pipeline_con_scaler(base_model, scaler_path: str):
    # Si ya es Pipeline con scaler → úsalo tal cual
    if hasattr(base_model, "named_steps"):
        return base_model
    # Si no, intenta crear Pipeline(scaler → SVM)
    if not os.path.exists(scaler_path):
        raise FileNotFoundError(
            "El SVM no es Pipeline y no se encontró el scaler en PATH_SCALER.\n"
            f"PATH_SCALER actual: {scaler_path}"
        )
    scaler = joblib.load(scaler_path)
    pipe = Pipeline([("scaler", scaler), ("svc", base_model)])
    return pipe

def tomar_Xy(df, features, label_col):
    missing = [c for c in features if c not in df.columns]
    if missing:
        raise ValueError(f"Faltan columnas en el CSV de calibración: {missing}")
    X = df.loc[:, features]
    if label_col not in df.columns:
        raise ValueError(f"No existe la columna de etiqueta '{label_col}' en el CSV.")
    y = df[label_col].to_numpy()
    return X, y

Mounted at /content/drive
[1/6] Cargando schema.json…
   → 45 features leídas.
[2/6] Cargando CSV de calibración…
   → Calibración con 346 filas, 45 features.
[3/6] Cargando SVM base…
   → El SVM base tiene 4 clases: ['caerse', 'caminar', 'gradas', 'sentarse']
[4/6] Ensamblando Pipeline (scaler → SVM) si hace falta…
[5/6] Calibrando (Platt/sigmoid) con cv='prefit'…
   → Log-loss (calibración): 0.0889
[6/6] Guardando modelo calibrado…


/usr/local/lib/python3.12/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(


[OK] Guardado: /content/drive/MyDrive/Colab Notebooks/TESIS/FASE ONLINE/ARCHIVOS ONLINE/svm_calibrado.pkl


**4. Construcción del Calibrador**

In [ ]:
def main():
    os.makedirs(OUT_DIR, exist_ok=True)

    print("[1/6] Cargando schema.json")
    FEATURES, schema = cargar_features_desde_schema(PATH_SCHEMA)
    print(f"   → {len(FEATURES)} features leídas.")

    print("[2/6] Cargando CSV de calibración")
    df = pd.read_csv(PATH_CALIB_CSV)

    # Filtros opcionales (si esas columnas existen)
    if FILTRO_UBICACION and "ubicacion" in df.columns:
        df = df[df["ubicacion"] == FILTRO_UBICACION]
    if FILTRO_FREC and "frecuencia_nominal" in df.columns:
        df = df[df["frecuencia_nominal"] == FILTRO_FREC]

    # Quitamos filas sin etiqueta
    if LABEL_COL in df.columns:
        df = df.dropna(subset=[LABEL_COL])

    if len(df) == 0:
        raise ValueError("El CSV de calibración no tiene filas después de filtrar/limpiar.")

    X_calib, y_calib = tomar_Xy(df, FEATURES, LABEL_COL)
    print(f"   → Calibración con {X_calib.shape[0]} filas, {X_calib.shape[1]} features.")

    print("[3/6] Cargando SVM base")
    svm_base = joblib.load(PATH_SVM_BASE)

    # Mostrar clases del SVM base
    if hasattr(svm_base, "classes_"):
        base_classes = list(svm_base.classes_)
    elif hasattr(svm_base, "named_steps"):
        for k in ["svc", "svm"]:
            if k in svm_base.named_steps and hasattr(svm_base.named_steps[k], "classes_"):
                base_classes = list(svm_base.named_steps[k].classes_)
                break
    if base_classes is not None:
        print(f"   → El SVM base tiene {len(base_classes)} clases: {base_classes}")

    print("[4/6] Ensamblando Pipeline (scaler → SVM)")
    model_prefit = armar_pipeline_con_scaler(svm_base, PATH_SCALER)

    print("[5/6] Calibrando (Platt/sigmoid) con cv='prefit'")
    calib = CalibratedClassifierCV(model_prefit, method="sigmoid", cv="prefit")
    calib.fit(X_calib, y_calib)

    # Cálculo Logg-loss
    try:
        proba = calib.predict_proba(X_calib)
        ll = log_loss(y_calib, proba, labels=calib.classes_)
        print(f"   → Log-loss (calibración): {ll:.4f}")
    except Exception as e:
        print("   (Aviso) No se pudo calcular log-loss:", e)

    print("[6/6] Guardando modelo calibrado…")
    joblib.dump(calib, PATH_OUT_CALIB)
    print(f"[OK] Guardado: {PATH_OUT_CALIB}")

if __name__ == "__main__":
    main()
